# Finding Eigenvalues efficiently (Suzuki - Varga pg 28)

## Import and complete eigenvalues problem

In [1]:
import numpy as np
import h5py
import math
import scipy as sp
from scipy import linalg
from scipy import optimize
from matplotlib import pyplot as plt
from IPython import get_ipython
get_ipython().run_line_magic('matplotlib', 'qt')
%matplotlib qt

A generic hamiltonian $h$ has been chosen

In [2]:
ha = np.array([[6.17291e-08, 2.91474e-08, 7.92295e-08],[2.91474e-08, 1.62907e-08, 3.54148e-08],[2.,3.,1.]])
ha = ha + ha.transpose()
print('Our full hamiltonian: \n', ha)

Our full hamiltonian: 
 [[1.23458200e-07 5.82948000e-08 2.00000008e+00]
 [5.82948000e-08 3.25814000e-08 3.00000004e+00]
 [2.00000008e+00 3.00000004e+00 2.00000000e+00]]


Three random non orthonormal basis have been chosen, and so their scalar product has been calculated

In [3]:
aa0 = np.array([1,2,1])
aa1 = np.array([0,1,1])
aa2 = np.array([1,1,1])
base = np.array([aa0,aa1,aa2])
base = base.transpose()
ga = np.zeros((3,3))
for i in range(3) :
    for j in range(3) :
        ga[i,j] = np.dot(base[:,i].transpose(),base[:,j])
print('Our scalsr prod matrix is: \n', ga)
np.tensordot(aa0,aa0.transpose(),axes=0)

Our scalsr prod matrix is: 
 [[6. 3. 4.]
 [3. 2. 2.]
 [4. 2. 3.]]


array([[1, 2, 1],
       [2, 4, 2],
       [1, 2, 1]])

We need to solove the generalized eigenvalues problem
$$H |\psi> = E N |\psi>$$

In [4]:
ea,wa = sp.linalg.eig(ha,ga)
print('The eigenvalues we should get are: \n', ea)

The eigenvalues we should get are: 
 [ 4.19615236e+00+0.j  2.08427614e-08+0.j -6.19615250e+00+0.j]


Now step by step procedure.

## First step: just a number

A random gaussian basis element is genereted $|A_0>$. **This basis is not orthonormal**

In [5]:
a = np.zeros((3,3))
a[0] = np.array([1,2,1])
print('The old basis is: \n',a[0])

The old basis is: 
 [1. 2. 1.]


The first element of our hamiltonian $h$ is calculated in this basis: $h_{00} = <A_0 | h |A_0>$

In [ ]:
ho = np.zeros((3,3))
ho[0,0] = 4
print('The original hamiltoniana at this first step: \n', ho)
g = np.zeros((3,3))
g[0,0] = a[0]@a[0].transpose()
print('The scalar product matrix at this first step: \n', g)

Our eigenvalues problem looks as
$$ <A_0 | h |A_0> = E <A_0 | A_0> \rightarrow h_{00} = \epsilon_0 g_{00}$$
where $g$ indicates the matrix of the scalar product $g_{ij} = <A_i |A_j>$. 

In this simple case our eignavalues problem is just an algebric equation which solution sipmly is 
$$ \epsilon_0 = \frac{h_{00}}{g_{00}}$$

In [ ]:
eigs = np.zeros(3)
eigs[0] = ho[0,0]/g[0,0]
print('First eigenvalues found: \n', eigs)

It's possible to solve this eigenvalues problem by redefining the basis vector in order to have an orthonormal basis. In this case we simple need to
$$|A_0> → |\phi_0> = \frac{|A_0>}{\sqrt{<A_0|A_0>}} = \frac{|A_0>}{\sqrt{g_{00}}}$$
With this new basis element the matrix element of our hamiltonian becomes 
$$<\phi_0 | h |\phi_0> = \epsilon_0$$

In [ ]:
phi = np.zeros((3,3))
phi[0] = a[0]/np.sqrt(g[0,0])
print('The new basis is: \n', phi[0])

We define a new hamiltonian matrix which basis elements are alculated in the new basis

In [ ]:
hn = np.zeros((3,3))
hn[0,0] = ho[0,0]/g[0,0]
print('The new hamiltoniana at the end of this first step: \n', hn)

It's useful to introduce a matrix $c_{ij}$ consisting of the coefficients to pass from the basis of gaussian $|A>$  (non-orthonormal) to the orthonormal basis $|\phi>$
$$ |\phi_i>  = \sum_{j} c_{ji} |A_j>$$

In [ ]:
c = np.zeros((3,3))
n = np.sqrt(g[0,0] - (c[0,0]*g[0,0]))
c[0,0] = 1/n
print('The matrix of basis conversion at this stage is: \n',c)

Finally it's useful to introduce the matrix of scalar product between the old basis and the new one
$$f_{ij} = <\phi_i|A_j>$$

In [ ]:
f = np.zeros((3,3))
f[0,0] = phi[0]@a[0].transpose()
# fm = np.zeros((3,3))
# fm[0,0] = c[0,0]*g[0,0]
print('Scalar product between old basis and new one: \n',f)

## Second step

### Adding another basis element, calculation of useful matrix and orthonormalization of gaussian basis

Adding of another random basis element $|A_1>$ and calculation of the new elements of the scalra product matrix:$g_{01} = g_{10} = <A_0 |A_1>$ and $g_{11} = <A_1 |A_1>$

In [ ]:
a[1] = np.array([0,1,1])
g[0,1] = a[0]@a[1].transpose()
g[1,0] = g[0,1]
g[1,1] = a[1]@a[1].transpose()
print('New scalar product matrix at this second step: \n', g)

Calculation of the others matrix elements with this new basis element: $h_{01} = h_{10} = <A_0 | h |A_1>$ and $h_{11} =<A_1 | h |A_1>$

In [ ]:
ho[0,1] = 4
ho[1,0] = ho[0,1]
ho[1,1] = 2
print('New original hamiltoniana at this second step: \n', ho)

Before to orthonormalize this basis is useful to calculate the first row remaining elements of the matrix $f_{ij}$ and $p_{ij}$:
$$ f_{01} = <\phi_0|A_1> \\ p_{01} = <\phi_0| h |A_1> $$

In [ ]:
f[0,1] = phi[0]@a[1].transpose()

The Gram-Scmihdt orthonormalization is applied to non-orthonormal gaussian basis:
$$|A_1> → |\phi_1> = \frac{|A_1> -|\phi_0><\phi_0|A_1>}{\sqrt{<A_1|A_1> - |<\phi_0|A_1>|^2 }} = \frac{|A_1> -|A_0>f_{01}}{\sqrt{g_{11} - f_{01}^2}}$$

In [ ]:
phi[1] = (a[1] - phi[0]*f[0,1])/np.sqrt(g[1,1]- f[0,1]**2)
print('The new orthonormal basis element is: \n', phi)

Now the other elements of the matrix $f_{ij}$

In [ ]:
f[1,0] = phi[1]@a[0].transpose()
f[1,1] = phi[1]@a[1].transpose()
('The matrix fij at this stage: \n',f)

Now the elements of the transformation matrix $c_{ij}$

In [ ]:
tmp=0
for i in range(1) :
    tmp += f[i,1]**2
n = np.sqrt(g[1,1] - tmp)
c[1,1] = 1/n
for i in range(1) :
    for j in range(1) :
        c[i,1] -= c[i,j]*f[j,1]
    c[i,1] /= n
c

And now the matrix elements of new hamiltonian within this new basis set 

In [ ]:
hn = c.transpose()@ho@c
print('The new hamiltoniana at the end of this second step: \n', hn)

### Diagonalization of the hamiltoniana and introduction of new basis orthonormal for both hamiltoniana and scalar product

In [ ]:
et,wt = sp.linalg.eig(hn)
print('The eigenvalues of the hamiltonian at this stage are: \n', et)
print('The eigenvectors of the hamiltonian at this stage are: \n', wt)

The problem it's been solved is:

1. Orthonormalization of gaussian basis in order to get a matrix of scalar product (the $g_{ij}$) equals to the identity matrix; the transformation has been called $c_{ij}$
$$ g_{ij} → c^T g c = 1$$

2. Projection of the old hamiltonian onto space spanned by this new basis
$$h_{ij} = <A_i|h|A_j> → h_{new} = c^T h c$$

3. Diagonalization of this last hamiltonian within its eigenvectors
$$h_{new} → h_{brand_{new}} = w^T h_{new} w = w^T c^T h c w$$

Then the final transformation to get both a diagonal hamiltonian and a identity scalar product is $c\dot w$

In [ ]:
c@wt

We can define the new basis $\phi_{new}$ at the end of this second step

In [ ]:
phin = wt.transpose()@phi
print('The new basis that makes both orthonormal the scalar product and diagonal the hamiltonian is:\n', phin)

Finally the new ransformation matrix $c_{ij}$ and the matrix elements of the hamiltonian within it

In [ ]:
cn = c@wt
hnd = cn.transpose()@ho@cn
print('The hamiltonian at the end of this first step:\n',hnd)

### Suzuki - Varga pg 28 procedure

This procedure is useful for refinement: after having a quasi diagonal matrix at stage $k$ we colud test $n$ new random basis element $|A_{k+1}>$ and then to choose the one that gives the lowest eigenvalue (without to diagonalize the matrix again). At thi stage we can find the eigenvalues of the new hamiltonian by the following equation
$$\epsilon_1 - h_{11} = \frac{h_{01}^2}{\epsilon_1 - \epsilon_0}$$
First of all a plot of this functions (both the rhs and lhs)

In [ ]:
x = np.linspace(-5,5,1000)
plt.grid()
plt.ylim(-10,10)
plt.plot(x,x-hn[1,1])
plt.plot(x,hn[0,1]**2/(x-hn[0,0]))
plt.show()

The intersections of the blu line with the orange one are our new eigenvalues of the hamiltonian. Managing the last equation this is equivalent to solve
$$\epsilon_1^2 - \epsilon_1(h_{00} + h_{11}) + h_{00}h_{11} - h_{01}^2 = 0$$
So let's define this equation first and then to solve it

In [ ]:
def newEig(x):
    f = x**2 - x*(hn[0,0]+hn[1,1]) + hn[0,0]*hn[1,1] - hn[0,1]**2
    return f
sol = sp.optimize.root(newEig,-2.)
eigs[0] = sol.x
sol = sp.optimize.root(newEig,1.)
eigs[1] = sol.x
eigs

**Exactly the same eigenvalues of the new hamiltonian!!** The new diagonalized hamiltonian at the end of this step is

## Third step

Adding of another random basis element $|A_2>$ and calculation of the new elements of the scalar product matrix

In [ ]:
a[2] = np.array([1,1,1])
for i in range(3):
    for j in range(3):
        g[i,j] = a[i].transpose()@a[j]
print('New scalar product matrix at this third step: \n', g)

Calculation of the others matrix elements in this new basis

In [ ]:
ho[0,2] = 5
ho[2,0] = ho[0,2]
ho[1,2] = 5
ho[2,1] = ho[1,2]
ho[2,2] = 2
print('New hamiltoniana at this third step: \n', ho)

As before we calculate the first and second row elements of the matrix $f_{ij}$ by using the new basis $|\phi_{new}>$
$$f_{0i} = <\phi_{new 0}|A_i> \\ f_{1i} = <\phi_{new 1}|A_i> $$

In [ ]:
f[0,0] = phin[0]@a[0].transpose()
f[0,1] = phin[0]@a[1].transpose()
f[0,2] = phin[0]@a[2].transpose()
f[1,0] = phin[1]@a[0].transpose()
f[1,1] = phin[1]@a[1].transpose()
f[1,2] = phin[1]@a[2].transpose()
print('The matrix fij is: \n',f)

We found the new matrix $c_{ij}$ to project the new element onto the new orthonormal basis

In [ ]:
tmp=0
for i in range(2) :
    tmp += f[i,2]**2
n = np.sqrt(g[2,2] - tmp)
cn[2,2] = 1/n
for i in range(2) :
    for j in range(2) :
        cn[i,2] -= cn[i,j]*f[j,2]
    cn[i,2] /= n
print('The matrix cij at this second step is: \n', cn)

As before we applied the Gram-Scmidt orthonormalization procedure to gaussian basis but by using the new basis $|\phi_{new}>$. The generalized expression of the new basis element is
$$|A_{k+1}> → |\phi_{k+1}> = \frac{|A_{k+1}> -\sum_{i=0}^{k}|\phi_i><\phi_i|A_{k+1}>}{\sqrt{<A_{k+1}|A_{k+1}> - \sum_{i=0}^{k}|<\phi_i|A_{k+1}>|^2 }}$$

In [ ]:
tmp = 0
tmp2 = 0
for i in range(2):
    tmp += phin[i]*f[i,-1]
    tmp2 += f[i,-1]**2
phin[2] = (a[-1] - tmp)/np.sqrt(g[-1,-1] - tmp2)
print('The new orthonormal basis element is: \n', phin[2])
print('The complete orthonormal basis set is: \n ', phin)

The matrix $f_{ij} = <\phi_i|A_j>$ is

In [ ]:
f = phin@a.transpose()
print('The matrix fij is: \n',f)

Calculation of the hamiltonian elements in this new orthonormal basis set

In [ ]:
hnn = cn.transpose()@ho@cn
print('The new hamiltonian into the new basis element is: \n', hnn)

### Diagonalization of the hamiltoniana and introduction of new basis orthonormal for both hamiltoniana and scalar product

In [ ]:
et2, wt2 = sp.linalg.eig(hnn)
print('Eig vecs : \n ', wt2)
print('Eig vals : \n ', et2)

In [ ]:
phinn = wt2.transpose()@phin
print('The new basis that makes both orthonormal the scalar product and diagonal the hamiltonian is:\n', phinn)

Finally the new ransformation matrix $c_{ij}$ and the matrix elements of the hamiltonian within it

In [ ]:
cnn = cn@wt2
hnd = cnn.transpose()@ho@cnn
print('The hamiltonian at the end of this first step:\n',hnd)

### Suzuki - Varga pg 28 procedure

Finding the eigenvalues of the new hamiltonian by the following equation general for any eigenvalues
$$\epsilon_{k+1} - h_{k+1, k+1} = \sum_{i=0}^{k} \frac{h_{i,k+1}^2}{\epsilon_{k+1} - \epsilon_i}$$
First of all a plot of this functions (both the rhs and lhs)

In [ ]:
def rhsE(x):
    rhs = 0
    for i in range(2):
        rhs += hnn[i,-1]**2/(x - hnn[i,i])
    return rhs
x = np.linspace(-10,3,1000)
plt.grid()
plt.ylim(-5,15)
plt.plot(x,x-hnn[-1,-1])
plt.plot(x,rhsE(x))
plt.show()

The intersections of the blu line with the orange one are our new eigenvalues of the hamiltonian. 
So let's define this equation first and then to solve it

In [ ]:
def newEig2(x):
    f = x - hnn[-1,-1] - rhsE(x)
    return f
sol = sp.optimize.root(newEig2,-10.)
eigs[0] = sol.x
sol = sp.optimize.root(newEig2,-2.)
eigs[1] = sol.x
sol = sp.optimize.root(newEig2,2.)
eigs[2] = sol.x
eigs

# Example from the codes

In [ ]:
ham = np.array(([-0.00115019 , -0.00136814] , [-0.00136814 , 0.00461564]))
ham

In [ ]:
def rhsE(x):
    rhs = 0
    for i in range(1):
        rhs += ham[i,-1]**2/(x - ham[i,i])
    return rhs
x = np.linspace(ham[0,0] + 2*ham[0,0],ham[-1,-1],1000)
plt.grid()
plt.ylim(-0.01,0.01)
plt.plot(x,x-ham[-1,-1])
plt.plot(x,rhsE(x))
plt.show()

In [ ]:
def newEig2(x):
    f = x - ham[-1,-1] - rhsE(x)
    return f
sol = sp.optimize.root(newEig2,ham[0,0] + 0.8*ham[0,0])
print(sol.x)

# SVD to fix NaN

In [2]:
d = 60*np.random.random(3)
print(d)
alphas = 1/d**2
print(alphas)
aDiag = np.diag(alphas)
aDiag

[ 9.67645667 30.23378741 29.81403807]
[0.0106799  0.00109399 0.00112502]


array([[0.0106799 , 0.        , 0.        ],
       [0.        , 0.00109399, 0.        ],
       [0.        , 0.        , 0.00112502]])

In [3]:
U = np.array([[1.,-1.,0],[1/2,1/2,1],[1/3,1/3,1/3]])
U

array([[ 1.        , -1.        ,  0.        ],
       [ 0.5       ,  0.5       ,  1.        ],
       [ 0.33333333,  0.33333333,  0.33333333]])

In [4]:
U1 = np.linalg.inv(U)
print(U1)
print(U1[0,:])
U1[2,:]

[[ 0.5 -1.   3. ]
 [-0.5 -1.   3. ]
 [-0.   2.  -3. ]]
[ 0.5 -1.   3. ]


array([-0.,  2., -3.])

In [5]:
pairs = np.array([[0,1],[0,2],[1,2]])
W = np.zeros((3,2))
for i in range(3):
    for j in range(2):
        W[i,j] = U1[pairs[i,0],j] - U1[pairs[i,1],j]       
WT = np.array([U1[0,:] - U1[1,:], U1[0,:] - U1[2,:], U1[1,:] - U1[2,:]])
print(W)
print(WT)

[[ 1.   0. ]
 [ 0.5 -3. ]
 [-0.5 -3. ]]
[[ 1.   0.   0. ]
 [ 0.5 -3.   6. ]
 [-0.5 -3.   6. ]]


In [6]:
A = np.transpose(W)@aDiag@W
print(A)

[[1.12346548e-02 4.65320138e-05]
 [4.65320138e-05 1.99710818e-02]]


In [7]:
g = np.zeros((3,3))
g[0,0] = 1/4/np.pi*((2*np.pi)**2/np.linalg.det(A+A))**1.5
print(g)

[[734182.54896864      0.              0.        ]
 [     0.              0.              0.        ]
 [     0.              0.              0.        ]]


In [8]:
d1 = 60*np.random.random(3)
#d1 = np.array([0.95315 , 0.95130, 0.6619])
print(d1)
alphas1 = 1/d1**2
print(alphas1)
a1Diag = np.diag(alphas1)
a1Diag
A1 = np.transpose(W)@a1Diag@W
print(A1)

[10.64262973 57.88051156 16.25791776]
[0.00882881 0.00029849 0.00378329]
[[0.00984926 0.0052272 ]
 [0.0052272  0.03673609]]


In [9]:
g[0,1] = 1/4/np.pi*((2*np.pi)**2/np.linalg.det(A+A1))**1.5
g[1,1] = 1/4/np.pi*((2*np.pi)**2/np.linalg.det(A1+A1))**1.5
g[1,0] = g[0,1]
print(g)

[[734182.54896864 494628.16058731      0.        ]
 [494628.16058731 403316.91679495      0.        ]
 [     0.              0.              0.        ]]


In [14]:
# d2 = np.random.random(3)
d2 = np.array([50.58493803, 16.05142796, 42.9746023])
print(d2)
alphas2 = 1/d2**2
print(alphas2)
a2Diag = np.diag(alphas2)
a2Diag
A2 = np.transpose(W)@a2Diag@W
print(A2)

[50.58493803 16.05142796 42.9746023 ]
[0.0003908  0.00388126 0.00054147]
[[ 0.00149649 -0.00500968]
 [-0.00500968  0.03980458]]


In [15]:
g[0,2] = 1/4/np.pi*((2*np.pi)**2/np.linalg.det(A+A2))**1.5
g[1,2] = 1/4/np.pi*((2*np.pi)**2/np.linalg.det(A1+A2))**1.5
g[2,2] = 1/4/np.pi*((2*np.pi)**2/np.linalg.det(A2+A2))**1.5
g[2,0] = g[0,2]
g[2,1] = g[1,2]
print(g)

[[   96967.41745188   334889.10638514   334889.1063963 ]
 [  334889.10638514 12192033.33055    12192033.33098375]
 [  334889.1063963  12192033.33098375 12192033.33141758]]


In [16]:
g1 = np.linalg.inv(g)
print(g1)

[[ 1.13935796e-05 -1.32039977e-04  1.31666719e-04]
 [-1.31924909e-04  1.53391689e+07 -1.53391689e+07]
 [ 1.31611952e-04 -1.53391689e+07  1.53391689e+07]]


In [17]:
ha = np.array([[1.34, 2.9, 7.92],[2.9, 1.63, 4.37],[7.92,4.37,5.81]])
print(ha)

[[1.34 2.9  7.92]
 [2.9  1.63 4.37]
 [7.92 4.37 5.81]]


In [18]:
H = g1@ha
e,w = np.linalg.eig(H)
print(e)

[-1.99409196e+07  2.27664739e-04 -1.13448291e-07]


In [26]:
U,S,V = sp.linalg.svd(g)
print(U)
S = np.diag(S)
print(S)
D,W = np.linalg.eig(g)
print(np.diag(D))
print(W)

W@np.transpose(W)
# S1 = np.zeros((3,3))
# S1[0:2,0:2] = 1/S[0:2,0:2]
# S1[0,1] = 0.
# S1[1,0] = 0.
# print(S1)

[[-1.94891468e-02  9.99810069e-01  6.07428966e-12]
 [-7.06972479e-01 -1.37809078e-02 -7.07106781e-01]
 [-7.06972479e-01 -1.37809078e-02  7.07106781e-01]]
[[2.43932986e+07 0.00000000e+00 0.00000000e+00]
 [0.00000000e+00 8.77355122e+04 0.00000000e+00]
 [0.00000000e+00 0.00000000e+00 3.25979531e-08]]
[[2.43932986e+07 0.00000000e+00 0.00000000e+00]
 [0.00000000e+00 8.77355122e+04 0.00000000e+00]
 [0.00000000e+00 0.00000000e+00 3.29272249e-08]]
[[-1.94891468e-02 -9.99810069e-01  6.07498804e-12]
 [-7.06972479e-01  1.37809078e-02 -7.07106781e-01]
 [-7.06972479e-01  1.37809078e-02  7.07106781e-01]]


array([[ 1.00000000e+00, -7.74410041e-16,  3.84097105e-16],
       [-7.74410041e-16,  1.00000000e+00, -5.55111512e-17],
       [ 3.84097105e-16, -5.55111512e-17,  1.00000000e+00]])

In [20]:
gt1 = U@S1@V
print(gt1)
gt1 = np.linalg.pinv(g,hermitian=True)
print(gt1)
et,wt = np.linalg.eig(gt1@ha)
et

[[ 1.13935796e-05 -1.56478644e-07 -1.56478644e-07]
 [-1.56478644e-07  2.26542603e-08  2.26542603e-08]
 [-1.56478644e-07  2.26542603e-08  2.26542603e-08]]
[[ 1.13935796e-05 -1.21513903e-04  1.21200945e-04]
 [-1.21513903e-04  1.41281819e+07 -1.41281819e+07]
 [ 1.21200945e-04 -1.41281819e+07  1.41281819e+07]]


array([-1.83666365e+07,  2.28507734e-04, -1.18136158e-07])

In [2]:
A = np.array([[1.,-2.4,-7.],[-2.4,-5.1,6.7],[-7.,6.7,-9.2]])
B = np.array([[3.,7.,7.],[7.,6.,6.],[7.,6.,5.99999999999999]])
print('A:\n', A)
print('B:\n', B)
print('Eigenvalues B:\n', np.linalg.eig(B)[0])
print('Det(B):\n',np.linalg.det(B))
print('Binv:\n', np.linalg.inv(B))
l,u = np.linalg.eig(np.linalg.inv(B)@A)
print('Eigenvec Binv A:\n', u)
print('Eigenval Binv A:\n', l)

A:
 [[ 1.  -2.4 -7. ]
 [-2.4 -5.1  6.7]
 [-7.   6.7 -9.2]]
B:
 [[3. 7. 7.]
 [7. 6. 6.]
 [7. 6. 6.]]
Eigenvalues B:
 [ 1.83742816e+01 -3.37428159e+00 -4.39625888e-15]
Det(B):
 3.0286884111774275e-13
Binv:
 [[-1.93548387e-01  2.32142857e-01  0.00000000e+00]
 [ 2.25806452e-01 -1.02354537e+14  1.02354537e+14]
 [-0.00000000e+00  1.02354537e+14 -1.02354537e+14]]
Eigenvec Binv A:
 [[ 9.05227861e-16  8.51984154e-01 -7.37791406e-01]
 [-7.07106781e-01 -2.71274558e-01 -6.26444066e-01]
 [ 7.07106781e-01 -4.47809239e-01 -2.51459089e-01]]
Eigenval Binv A:
 [ 2.83522067e+15 -2.05124864e+00 -3.69657338e-01]


In [7]:
D,U = np.linalg.eig(B)
print('Eigenvec B:\n',U)
print('Eigenval B:\n',np.diag(D))
print('Pseudoinv B:\n',np.linalg.pinv(B))
l,u = np.linalg.eig(np.linalg.pinv(B)@A)
print('Eigenval PinvB A:\n',l)
PinvD =np.zeros((3,3))
for i in range(2):
    PinvD[i,i] = 1/np.diag(D)[i,i]
PinvD[2,2] = 0.
print('Pseudoinv of Eig matrix of B:\n', PinvD)
print('Pseudoinv B code :\n', U@PinvD@np.transpose(U))

Eigenvec B:
 [[-5.41377668e-01 -8.40779531e-01  7.30598377e-16]
 [-5.94520908e-01  3.82811820e-01 -7.07106781e-01]
 [-5.94520908e-01  3.82811820e-01  7.07106781e-01]]
Eigenval B:
 [[ 1.83742816e+01  0.00000000e+00  0.00000000e+00]
 [ 0.00000000e+00 -3.37428159e+00  0.00000000e+00]
 [ 0.00000000e+00  0.00000000e+00 -4.39625888e-15]]
Pseudoinv B:
 [[-0.19354839  0.11290323  0.11290323]
 [ 0.11290323 -0.02419355 -0.02419355]
 [ 0.11290323 -0.02419355 -0.02419355]]
Eigenval PinvB A:
 [-1.91930262e+00 -3.75052221e-01 -1.17763742e-16]
Pseudoinv of Eig matrix of B:
 [[ 0.0544239   0.          0.        ]
 [ 0.         -0.29635938  0.        ]
 [ 0.          0.          0.        ]]
Pseudoinv B code :
 [[-0.19354839  0.11290323  0.11290323]
 [ 0.11290323 -0.02419355 -0.02419355]
 [ 0.11290323 -0.02419355 -0.02419355]]


In [31]:
U,D,V = sp.linalg.svd(B)
print('Eigenvec B:\n',U)
print('Eigenval B:\n',np.diag(D))

Eigenvec B:
 [[-5.87388664e-01  8.09304984e-01  1.02787791e-15]
 [-5.72265043e-01 -4.15346507e-01 -7.07106781e-01]
 [-5.72265043e-01 -4.15346507e-01  7.07106781e-01]]
Eigenval B:
 [[1.52654816e+01 0.00000000e+00 0.00000000e+00]
 [0.00000000e+00 1.99124856e+00 0.00000000e+00]
 [0.00000000e+00 0.00000000e+00 4.35942595e-15]]


In [3]:
H = np.array([[ -3580.75 ,-36700.7],[-36700.7  ,35026.2]])
PinvG = np.array([[0.000000649599 , -0.000000061265],[-0.000000061265  , 0.000000020147]])
l,w = np.linalg.eig()

In [19]:
f = h5py.File("2BodySVD_8.h5")
list(f)

['A',
 'alphas',
 'cij',
 'coulombPotential',
 'fourBodyPotential',
 'harmonicTrap',
 'isospins',
 'kinetic',
 'overlaps',
 'phiij',
 'radiusSquare',
 'spins',
 'threeBodyPotential',
 'threeBodyPotentialSpin',
 'twoBodyPotential',
 'u']

In [20]:
k = f['kinetic'][:]

In [21]:
p = f['twoBodyPotential'][:]

In [22]:
h = k + p
h.shape

(6, 6)

In [24]:
overlap = f['overlaps'][:]
overlap.shape

(6, 6)

In [25]:
cij = f['cij'][:]
cij.shape

(6, 6)

In [26]:
cij@np.transpose(cij)

array([[ 2.88356944e-03,  7.18265623e-06,  4.97409914e-05,
         1.47589663e-04,  5.59129765e-05,  5.36180290e-02],
       [ 7.18265623e-06,  1.46526500e-03, -4.10426952e-05,
         1.53513363e-05,  2.47362591e-05, -6.16300640e-05],
       [ 4.97409914e-05, -4.10426952e-05,  5.71459749e-04,
         3.22293393e-06, -1.82379291e-05,  6.42469428e-05],
       [ 1.47589663e-04,  1.53513363e-05,  3.22293393e-06,
         1.82430498e-04,  9.10298083e-06,  2.25768478e-03],
       [ 5.59129765e-05,  2.47362591e-05, -1.82379291e-05,
         9.10298083e-06,  5.69106871e-04,  8.31562435e-04],
       [ 5.36180290e-02, -6.16300640e-05,  6.42469428e-05,
         2.25768478e-03,  8.31562435e-04,  9.99999197e-01]])

In [27]:
sp.linalg.eig(h,overlap)

(array([2.54561116e-01+0.j, 4.90520976e-03+0.j, 2.98357423e-03+0.j,
        1.47075979e-03+0.j, 6.60148786e-04+0.j, 1.59913351e-04+0.j,
        6.70788299e-06+0.j]),
 array([[ 1.51864104e-01, -1.83028810e-01,  1.73835114e-01,
         -1.60249842e-01, -1.35369878e-01,  1.37930720e-01,
         -1.89374721e-01],
        [-3.91409759e-01,  4.71922387e-01, -5.94853469e-01,
          6.17738242e-01,  6.68585748e-01, -6.54224093e-01,
          2.85583024e-01],
        [ 3.57433453e-01, -4.30662995e-01,  5.47069317e-01,
         -5.70949110e-01, -6.23576282e-01,  6.11299733e-01,
         -2.52071273e-01],
        [ 3.85781135e-01, -4.52948553e-01,  3.29630935e-01,
         -3.04280024e-01, -2.20808234e-01,  2.48352964e-01,
         -5.58226694e-01],
        [-5.02958661e-01,  5.94267676e-01, -4.56056435e-01,
          4.17310299e-01,  3.11548649e-01, -3.42924326e-01,
          7.11778047e-01],
        [-5.42385954e-01, -1.62401543e-03, -1.35677582e-03,
         -1.53195239e-03,  1.37013494e-